# fixer

In [9]:
# pip install requests pandas openpyxl
import requests
import pandas as pd
from datetime import datetime
from pathlib import Path

# === CONFIG ===
ACCESS_KEY = "4e24104d947d9a92ecad3d7c44059f9f"
CURRENCIES = [
    "JOD","USD","SAR","EUR","JPY","AED","AUD","GBP","INR","EGP","CAD","IRR",
    "QAR","CHF","SEK","OMR","KWD","ZAR","CNY","SGD","LYD","NZD","KRW","NOK",
    "BHD","TRY","DKK","THB","IQD", "SZL"
]
FIXER_URL = "https://data.fixer.io/api/latest"
OUTPUT_XLSX = Path("fixer/fx_matrix_all.xlsx")

def fetch_latest_rates(access_key: str, symbols: list[str]) -> dict:
    """
    Get latest rates from Fixer (EUR is the base on free plan).
    Returns dict of currency -> rate_vs_EUR with EUR=1.0 included.
    """
    params = {"access_key": access_key, "symbols": ",".join(sorted(set(symbols)))}
    r = requests.get(FIXER_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()

    if not data.get("success", False):
        raise RuntimeError(f"Fixer error: {data.get('error', data)}")

    rates = data.get("rates", {})
    # Ensure EUR present as 1.0 (EUR is the base on free tier)
    rates["EUR"] = 1.0
    out = {c: rates[c] for c in symbols if c in rates}
    missing = [c for c in symbols if c not in out]
    if missing:
        raise RuntimeError(f"Missing rates for: {missing}. Check symbol support on your plan/account.")
    return out, data.get("date", ""), data.get("timestamp", None)

def build_cross_matrix(rates_vs_eur: dict[str, float], order: list[str]) -> pd.DataFrame:
    """
    cross rate A->B = (EUR->B)/(EUR->A) = rates_vs_eur[B] / rates_vs_eur[A]
    """
    df = pd.DataFrame(index=order, columns=order, dtype=float)
    for a in order:
        for b in order:
            if a == b:
                df.loc[a, b] = 1.0
            else:
                df.loc[a, b] = rates_vs_eur[b] / rates_vs_eur[a]
    return df.round(6)

def main():
    rates_vs_eur, date_str, ts = fetch_latest_rates(ACCESS_KEY, CURRENCIES)
    matrix = build_cross_matrix(rates_vs_eur, CURRENCIES)

    base_series = pd.Series(rates_vs_eur, name="rate_vs_EUR").to_frame()

    stamp = date_str or datetime.utcnow().strftime("%Y-%m-%d")
    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as xw:
        matrix.to_excel(xw, sheet_name=f"matrix_{stamp}")
        base_series.to_excel(xw, sheet_name=f"vs_EUR_{stamp}")

    print(f"Saved FX matrix for {CURRENCIES} to {OUTPUT_XLSX.resolve()}")
    print("Sheet names:", f"matrix_{stamp}", f"vs_EUR_{stamp}")

if __name__ == "__main__":
    main()


Saved FX matrix for ['JOD', 'USD', 'SAR', 'EUR', 'JPY', 'AED', 'AUD', 'GBP', 'INR', 'EGP', 'CAD', 'IRR', 'QAR', 'CHF', 'SEK', 'OMR', 'KWD', 'ZAR', 'CNY', 'SGD', 'LYD', 'NZD', 'KRW', 'NOK', 'BHD', 'TRY', 'DKK', 'THB', 'IQD', 'SZL'] to D:\SAP-BOT - Currency - Test\fixer\fx_matrix_all.xlsx
Sheet names: matrix_2025-09-22 vs_EUR_2025-09-22


In [ ]:
# pip install pandas openpyxl
import pandas as pd
import json
from pathlib import Path

INPUT_XLSX = Path("fixer/fx_matrix_all.xlsx")
OUTPUT_JSON = Path("fixer/fx_payload.json")

VALID_FROM = "2025-09-07"  
EXCHANGE_RATE_TYPE = "M"
QUOTATION = "Direct"

def build_payload(xlsx_path: Path, sheet_name: str | None = None) -> list[dict]:
    # Auto-detect the matrix sheet if not provided
    xls = pd.ExcelFile(xlsx_path)
    if sheet_name is None:
        matrix_sheets = [s for s in xls.sheet_names if s.startswith("matrix_")]
        if not matrix_sheets:
            raise RuntimeError("No matrix sheet found in Excel")
        sheet_name = matrix_sheets[0]

    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, index_col=0)

    payload = []
    for from_cur in df.index:
        for to_cur in df.columns:
            if from_cur == to_cur:
                continue  # skip self-to-self
            rate = float(df.loc[from_cur, to_cur])
            payload.append({
                "ExchangeRateType": EXCHANGE_RATE_TYPE,
                "FromCurrency": from_cur,
                "ToCurrency": to_cur,
                "ValidFrom": VALID_FROM,
                "Quotation": QUOTATION,
                "ExchangeRate": rate
            })
    return payload

def main():
    payload = build_payload(INPUT_XLSX)

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=4)

    print(f"Saved {len(payload)} entries to {OUTPUT_JSON.resolve()}")

if __name__ == "__main__":
    main()


Saved 870 entries to D:\SAP-BOT - Currency - Test\fixer\fx_payload.json


# All Currencies

# Bot

In [5]:
import os
import re
import json
import requests

def main():
    url = "http://127.0.0.1:8000/currency/exchange-rates/batch"

    # --- locate latest run dir and load the produced JSON ---
    base_dir = "WebService"
    run_dir = _pick_latest_run_dir(base_dir)
    if not run_dir:
        raise SystemExit(f"No run dir under {base_dir} matching YYYY-MM-DD__YYYY-MM-DD")

    payload_path = os.path.join(run_dir, "exchange_rates_payload.json")
    if not os.path.isfile(payload_path):
        raise SystemExit(f"Payload not found: {payload_path}")

    with open(payload_path, "r", encoding="utf-8") as f:
        payload = json.load(f)

    # keep your structure exactly, but normalize dates to YYYY-MM-DD like your example
    payload = [_normalize_item_dates_yyyy_mm_dd(x) for x in payload if isinstance(x, dict)]

    try:
        r = requests.post(url, json=payload, timeout=1000000)
        r.raise_for_status()
        print(json.dumps(r.json(), indent=2))
    except requests.RequestException as e:
        print(f"HTTP error: {e}")
        if getattr(e, "response", None) is not None:
            try:
                print("Response body:", e.response.text)
            except Exception:
                pass

def _pick_latest_run_dir(base_dir: str) -> str | None:
    pat = re.compile(r"^\d{4}-\d{2}-\d{2}__\d{4}-\d{2}-\d{2}$")
    if not os.path.isdir(base_dir):
        return None
    candidates = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and pat.match(name):
            try:
                candidates.append((p, os.path.getmtime(p)))
            except OSError:
                pass
    if not candidates:
        return None
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0]

def _normalize_item_dates_yyyy_mm_dd(item: dict) -> dict:
    """If ValidFrom is MM/DD/YYYY, convert to YYYY-MM-DD. Otherwise leave as-is."""
    v = str(item.get("ValidFrom", "")).strip()
    m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", v)  # MM/DD/YYYY
    if m:
        mm, dd, yyyy = m.group(1), m.group(2), m.group(3)
        item = dict(item)
        item["ValidFrom"] = f"{yyyy}-{mm}-{dd}"
    return item

if __name__ == "__main__":
    main()


{
  "ok": false,
  "workers": 2,
  "total": 591,
  "created": 512,
  "failed": 79,
  "skipped": 0,
  "interrupted_by_dialog": false,
  "results": [
    {
      "index": 1,
      "payload": {
        "ExchangeRateType": "M",
        "FromCurrency": "AED",
        "ToCurrency": "AUD",
        "ValidFrom": "07/01/2025",
        "Quotation": "Direct",
        "ExchangeRate": "0.41378"
      },
      "status": "created",
      "dialog_open": false,
      "dialog_text": "",
      "footer_clicks": 0,
      "intermediate_toasts": [],
      "messages": [],
      "popover_text": "",
      "worker": 1
    },
    {
      "index": 2,
      "payload": {
        "ExchangeRateType": "M",
        "FromCurrency": "AED",
        "ToCurrency": "CAD",
        "ValidFrom": "07/01/2025",
        "Quotation": "Direct",
        "ExchangeRate": "0.37057"
      },
      "status": "created",
      "dialog_open": false,
      "dialog_text": "",
      "footer_clicks": 0,
      "intermediate_toasts": [],
      "mess

In [6]:
import json
import requests

def main():
    url = "http://127.0.0.1:8000/currency/exchange-rates/batch"
    payload = [
        # JOD row
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.410359},
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 5.289174},
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.203183},
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 5.179547},
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 5.141297},
        {"ExchangeRateType": "M","FromCurrency": "JOD","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1842.820371},
        # USD row
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.709039},
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 3.750233},
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.853104},
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 3.672504},
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 3.645383},
        {"ExchangeRateType": "M","FromCurrency": "USD","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1306.632416},

        # SAR row
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.189065},
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.26665},
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.22748},
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.979273},
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.972042},
        {"ExchangeRateType": "M","FromCurrency": "SAR","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 348.413633},

        # EUR row
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.831129},
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.17219},
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 4.395986},
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 4.304872},
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 4.273081},
        {"ExchangeRateType": "M","FromCurrency": "EUR","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1531.621452},

        # AED row
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.193067},
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.272294},
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.021165},
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.232295},
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.992615},
        {"ExchangeRateType": "M","FromCurrency": "AED","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 355.787919},

        # QAR row
        {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.194503},
        {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Indirect","ExchangeRate": 0.27432},
        {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.028763},
        {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Indirect","ExchangeRate": 0.234023},
        {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 1.00744},
       # {"ExchangeRateType": "M","FromCurrency": "QAR","ToCurrency": "IQD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 358.434921},

        # IQD row
        {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "JOD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.000543},
        {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "USD","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.000765},
        {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "SAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.00287},
        {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "EUR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.000653},
        {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "AED","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.002811},
       # {"ExchangeRateType": "M","FromCurrency": "IQD","ToCurrency": "QAR","ValidFrom": "2025-09-08","Quotation": "Direct","ExchangeRate": 0.00279},

    ]

    try:
        r = requests.post(url, json=payload, timeout=1000000)
        r.raise_for_status()
        print(json.dumps(r.json(), indent=2))
    except requests.RequestException as e:
        print(f"HTTP error: {e}")
        if e.response is not None:
            try:
                print("Response body:", e.response.text)
            except Exception:
                pass

if __name__ == "__main__":
    main()


{
  "ok": false,
  "workers": 2,
  "total": 40,
  "created": 0,
  "failed": 40,
  "skipped": 0,
  "interrupted_by_dialog": false,
  "error": "batch_failed: TimeoutError: Element not found for ('xpath', \"//input[substring(@id,string-length(@id)-string-length('ExchangeRateTypeForEdit::Field-input-inner')+1)='ExchangeRateTypeForEdit::Field-input-inner']\")",
  "results": [],
  "batch_id": "20250923-151439-08e75634",
  "received": 40,
  "duration_sec": 190.73,
  "reports": {
    "dir": "reports\\20250923-151439-08e75634",
    "result_json": "reports\\20250923-151439-08e75634\\result.json",
    "failed_json": "reports\\20250923-151439-08e75634\\failed.json",
    "failed_csv": "reports\\20250923-151439-08e75634\\failed.csv"
  },
  "email": {
    "ok": false,
    "reason": "not_requested"
  }
}


In [ ]:
from babel.numbers import format_decimal

def format_for_ui5(rate: float, locale: str = "en_US") -> str:
    """
    Convert a float to a locale-aware string for UI5 input fields.
    Example: 0.7089 -> "0.709" (en_US) or "0,709" (de_DE)
    """
    return format_decimal(rate, locale=locale)

# Example usage
rate = float("0.708901")  # your parsed value
print(f"Raw rate: {rate}")  # → 0.7089
print(format_for_ui5(rate, "en_US"))  # → "0.709"
print(format_for_ui5(rate, "de_DE"))  # → "0,709"
print(format_for_ui5(rate, "ar_JO"))  # → "٠٫٧٠٩"


# Ghanem WebService

In [1]:
import os
import re
import time
import json
import csv
import socket
from datetime import date, timedelta
from typing import Iterable, Dict, Any, List, Optional, Tuple

import requests
from requests.exceptions import ConnectTimeout, ReadTimeout, ProxyError, SSLError, ConnectionError as ReqConnectionError

BASE_URL = "http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates"

# Timeouts: (connect_timeout, read_timeout)
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 30
MAX_RETRIES = 3
RETRY_BACKOFF_BASE = 2

DATE_RE = re.compile(r"^\d{2}/\d{2}/\d{4}$")
RUN_DIR: Optional[str] = None  # set by fetch_daily_range()
NO_PROXY = {"http": None, "https": None}

def validate_mmddyyyy(s: str) -> None:
    if not DATE_RE.match(s):
        raise ValueError(f"Date must be MM/DD/YYYY, got {s}")

def daterange_days(start: date, end: date) -> Iterable[date]:
    d = start
    while d <= end:
        yield d
        d += timedelta(days=1)

def mmddyyyy(d: date) -> str:
    return f"{d.month:02d}/{d.day:02d}/{d.year:04d}"

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _paths() -> Tuple[str, str]:
    """
    Returns (json_dir, csv_path) inside the current RUN_DIR.
    """
    assert RUN_DIR, "RUN_DIR is not set"
    json_dir = os.path.join(RUN_DIR, "exchange_rates_json")
    csv_path = os.path.join(RUN_DIR, "exchange_rates_agg.csv")
    return json_dir, csv_path

def save_day_json(d: date, obj: Any) -> str:
    json_dir, _ = _paths()
    ensure_dir(json_dir)
    p = os.path.join(json_dir, f"{d.isoformat()}.json")
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    return p

def flatten_rows(obj: Any) -> List[Dict[str, Any]]:
    if isinstance(obj, list):
        return [x if isinstance(x, dict) else {"value": x} for x in obj]
    if isinstance(obj, dict):
        if isinstance(obj.get("data"), list):
            return [x if isinstance(x, dict) else {"value": x} for x in obj["data"]]
        return [obj]
    return [{"value": obj}]

def append_csv(rows: List[Dict[str, Any]]) -> None:
    if not rows:
        return
    _, csv_path = _paths()

    # discover headers
    all_keys = set()
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            all_keys.update(rdr.fieldnames or [])
    for r in rows:
        all_keys.update(r.keys())
    fieldnames = sorted(all_keys)

    # load existing
    existing: List[Dict[str, Any]] = []
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            for r in rdr:
                existing.append(r)

    # write all
    ensure_dir(os.path.dirname(csv_path))
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in existing:
            w.writerow({k: r.get(k, "") for k in fieldnames})
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

def _tcp_preflight(host: str, port: int = 80, timeout: int = 3) -> bool:
    """Quick TCP connect preflight—returns True if TCP handshake succeeds."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except OSError:
        return False

def _make_session(disable_env_proxies: bool = True, force_no_proxy: bool = True) -> requests.Session:
    s = requests.Session()
    s.trust_env = not disable_env_proxies  # False → do NOT inherit env proxies
    if force_no_proxy:
        s.proxies.update(NO_PROXY)
    s.headers.update({"Content-Type": "application/json"})
    return s

# ------------------ Filters ------------------
_FRF = "FRF"

def _extract_pair_ci(row: Dict[str, Any]) -> Tuple[str, str]:
    """Extract (FROM, TO) in upper-case; returns ('','') if not found."""
    if not isinstance(row, dict):
        return "", ""
    lm = {k.lower(): k for k in row.keys()}
    def _get_one(keys: List[str]) -> str:
        for k in keys:
            real = lm.get(k)
            if real:
                v = row.get(real, "")
                return str(v).strip().upper() if isinstance(v, (str, int, float)) else ""
        return ""
    f = _get_one(["fromcurrency", "sourcecurrency", "from", "basecurrency"])
    t = _get_one(["tocurrency", "targetcurrency", "to", "quotecurrency", "targetcurrency"])
    return f, t

def _has_frf_pair(row: Dict[str, Any]) -> bool:
    f, t = _extract_pair_ci(row)
    return f == _FRF or t == _FRF

def _same_currency_pair(row: Dict[str, Any]) -> bool:
    """True if FromCurrency == ToCurrency (and both present)."""
    f, t = _extract_pair_ci(row)
    return bool(f) and f == t

def _filter_obj_exclude_pairs(obj: Any) -> Any:
    """
    Remove any row where:
      - FromCurrency or ToCurrency == 'FRF', OR
      - FromCurrency == ToCurrency
    Keeps unknown/non-dict shapes as-is.
    """
    if isinstance(obj, list):
        return [r for r in obj if not (isinstance(r, dict) and (_has_frf_pair(r) or _same_currency_pair(r)))]
    if isinstance(obj, dict) and isinstance(obj.get("data"), list):
        new_obj = dict(obj)
        new_obj["data"] = [r for r in obj["data"] if not (isinstance(r, dict) and (_has_frf_pair(r) or _same_currency_pair(r)))]
        return new_obj
    return obj
# ---------------------------------------------

def post_one_day(d: date, session: Optional[requests.Session] = None) -> Any:
    payload = {"FromDate": mmddyyyy(d), "ToDate": mmddyyyy(d)}
    validate_mmddyyyy(payload["FromDate"]); validate_mmddyyyy(payload["ToDate"])

    s = session or _make_session()
    last_err: Optional[Exception] = None

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"→ POST {BASE_URL} body={payload} (attempt {attempt}/{MAX_RETRIES})")
            r = s.post(
                BASE_URL,
                json=payload,
                timeout=(CONNECT_TIMEOUT, READ_TIMEOUT),
                allow_redirects=False,
            )
            print(f"   HTTP {r.status_code}")
            r.raise_for_status()
            try:
                return r.json()
            except ValueError:
                return {"raw": r.text}
        except (ConnectTimeout, ReadTimeout) as e:
            last_err = e
            print(f"   Timeout: {e}")
        except (ProxyError, SSLError, ReqConnectionError) as e:
            last_err = e
            print(f"   Connection/Proxy error: {e}")
        except Exception as e:
            last_err = e
            print(f"   Other error: {e}")

        time.sleep(min(RETRY_BACKOFF_BASE ** attempt, 8))

    raise RuntimeError(f"Failed after {MAX_RETRIES} attempts: {last_err}")

def fetch_daily_range(start: date, end: date) -> None:
    global RUN_DIR
    # Set per-run directory: WebService/YYYY-MM-DD__YYYY-MM-DD
    run_name = f"{start.isoformat()}__{end.isoformat()}"
    RUN_DIR = os.path.join("WebService", run_name)
    ensure_dir(RUN_DIR)
    print(f"[run-dir] {RUN_DIR}")

    # Preflight
    host = BASE_URL.split("://", 1)[1].split("/", 1)[0]
    if not _tcp_preflight(host, 80, timeout=3):
        raise RuntimeError(
            f"Cannot open TCP to {host}:80. Likely VPN route/firewall issue. "
            "Check that 192.168.0.0/24 is routed via VPN and outbound :80 is allowed."
        )

    session = _make_session(disable_env_proxies=True, force_no_proxy=True)

    for d in daterange_days(start, end):
        print(f"[fetch] {d.isoformat()} …")
        obj = post_one_day(d, session=session)

        # Apply filters (FRF pairs + same-currency pairs) before saving/aggregating
        obj_filtered = _filter_obj_exclude_pairs(obj)

        save_day_json(d, obj_filtered)
        rows = flatten_rows(obj_filtered)
        for r in rows:
            r.setdefault("_service_date", d.isoformat())
        append_csv(rows)

# === EXAMPLE RUN ===
if __name__ == "__main__":
    # Pick your date window here:
    fetch_daily_range(date(2025, 7, 1), date(2025, 7, 1))
    print("Done.")


[run-dir] WebService\2025-07-01__2025-07-01
[fetch] 2025-07-01 …
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'FromDate': '07/01/2025', 'ToDate': '07/01/2025'} (attempt 1/3)
   HTTP 200
Done.


In [2]:
import os
import re
import json
import csv
from typing import Dict, Any, List, Tuple, Set, Optional

BASE_DIR = "WebService"
EXCLUDED_FILE = os.path.join(BASE_DIR, "excluded_rates.txt")

RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}__\d{4}-\d{2}-\d{2}$")

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _norm_currency(s: str) -> str:
    return (s or "").strip().upper()

def _find_key_case_insensitive(d: Dict[str, Any], *candidates: str) -> str:
    lower_map = {k.lower(): k for k in d.keys()}
    for c in candidates:
        k = lower_map.get(c.lower())
        if k is not None:
            return k
    return ""

def _row_pair(row: Dict[str, Any]) -> Tuple[str, str]:
    if not isinstance(row, dict):
        return "", ""
    from_k = _find_key_case_insensitive(row, "FromCurrency", "From", "BaseCurrency", "SourceCurrency")
    to_k   = _find_key_case_insensitive(row, "ToCurrency", "To", "QuoteCurrency", "TargetCurrency")
    f = _norm_currency(row.get(from_k, "")) if from_k else ""
    t = _norm_currency(row.get(to_k, ""))   if to_k else ""
    return f, t

def load_excluded_pairs(path: str) -> Set[Tuple[str, str]]:
    pairs: Set[Tuple[str, str]] = set()
    if not os.path.isfile(path):
        print(f"[warn] excluded list not found: {path} (no pairs will be excluded)")
        return pairs
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#"):
                continue
            if "/" not in s:
                parts = s.replace(" ", "/").split("/")
            else:
                parts = s.split("/")
            if len(parts) != 2:
                continue
            a, b = _norm_currency(parts[0]), _norm_currency(parts[1])
            if a and b:
                pairs.add((a, b))
    return pairs

def should_keep(row: Dict[str, Any], excluded: Set[Tuple[str, str]]) -> bool:
    f, t = _row_pair(row)
    if not f or not t:
        return True  # fail-open on unknown shapes
    return (f, t) not in excluded

def filter_json_object(obj: Any, excluded: Set[Tuple[str, str]]) -> Any:
    try:
        if isinstance(obj, list):
            return [r for r in obj if not isinstance(r, dict) or should_keep(r, excluded)]
        if isinstance(obj, dict):
            if "data" in obj and isinstance(obj["data"], list):
                new_obj = dict(obj)
                new_obj["data"] = [r for r in obj["data"] if not isinstance(r, dict) or should_keep(r, excluded)]
                return new_obj
            return obj
        return obj
    except Exception:
        return obj

def pick_latest_run_dir(base_dir: str) -> Optional[str]:
    if not os.path.isdir(base_dir):
        return None
    candidates = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            candidates.append((p, os.path.getmtime(p)))
    if not candidates:
        return None
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0]

def filter_all_jsons(run_dir: str, excluded: Set[Tuple[str, str]]) -> Tuple[int, int]:
    json_in = os.path.join(run_dir, "exchange_rates_json")
    json_out = os.path.join(run_dir, "exchange_rates_json_filtered")
    ensure_dir(json_out)

    processed = 0
    written = 0

    if not os.path.isdir(json_in):
        print(f"[warn] JSON input dir not found: {json_in}")
        return (0, 0)

    for name in os.listdir(json_in):
        if not name.lower().endswith(".json"):
            continue
        inp = os.path.join(json_in, name)
        out = os.path.join(json_out, name)
        try:
            with open(inp, "r", encoding="utf-8") as f:
                obj = json.load(f)
            new_obj = filter_json_object(obj, excluded)
            with open(out, "w", encoding="utf-8") as f:
                json.dump(new_obj, f, ensure_ascii=False, indent=2)
            processed += 1
            written += 1
        except Exception as e:
            print(f"[json] skip {inp}: {e}")
            processed += 1
    return processed, written

def filter_csv(run_dir: str, excluded: Set[Tuple[str, str]]) -> Tuple[int, int]:
    csv_in = os.path.join(run_dir, "exchange_rates_agg.csv")
    csv_out = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")

    if not os.path.isfile(csv_in):
        print(f"[warn] CSV input not found: {csv_in}")
        return (0, 0)

    with open(csv_in, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        fieldnames = rdr.fieldnames or []
        rows = list(rdr)

    if not fieldnames:
        print(f"[warn] CSV has no header: {csv_in}")
        return (0, 0)

    kept: List[Dict[str, Any]] = []
    for r in rows:
        if should_keep(r, excluded):
            kept.append(r)

    if "FromCurrency" not in fieldnames:
        fieldnames = fieldnames + ["FromCurrency"]
    if "ToCurrency" not in fieldnames:
        fieldnames = fieldnames + ["ToCurrency"]

    with open(csv_out, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in kept:
            out_row = {k: r.get(k, "") for k in fieldnames}
            fcur, tcur = _row_pair(r)
            if fcur and not out_row.get("FromCurrency"):
                out_row["FromCurrency"] = fcur
            if tcur and not out_row.get("ToCurrency"):
                out_row["ToCurrency"] = tcur
            w.writerow(out_row)

    return (len(rows), len(kept))

# === RUN ===
if __name__ == "__main__":
    run_dir = pick_latest_run_dir(BASE_DIR)
    if not run_dir:
        raise SystemExit(f"No run directory found under {BASE_DIR} matching YYYY-MM-DD__YYYY-MM-DD")
    print(f"[run-dir] {run_dir}")

    excluded = load_excluded_pairs(EXCLUDED_FILE)

    j_in, j_out = filter_all_jsons(run_dir, excluded)
    print(f"[done] JSON files processed={j_in}, written={j_out} → {os.path.join(run_dir, 'exchange_rates_json_filtered')}")

    c_in, c_out = filter_csv(run_dir, excluded)
    print(f"[done] CSV rows in={c_in}, out={c_out} → {os.path.join(run_dir, 'exchange_rates_agg.filtered.csv')}")


[run-dir] WebService\2025-07-01__2025-07-01
[done] JSON files processed=1, written=1 → WebService\2025-07-01__2025-07-01\exchange_rates_json_filtered
[done] CSV rows in=784, out=591 → WebService\2025-07-01__2025-07-01\exchange_rates_agg.filtered.csv


In [3]:
import os
import re
import csv
import json
from typing import Optional, List, Dict, Any, Tuple

BASE_DIR = "WebService"
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}__\d{4}-\d{2}-\d{2}$")
FNAME_DATE_RE = re.compile(r"^(\d{4})-(\d{2})-(\d{2})\.json$", re.IGNORECASE)

# ---------- utils ----------
def pick_latest_run_dir(base_dir: str) -> Optional[str]:
    if not os.path.isdir(base_dir):
        return None
    candidates: List[Tuple[str, float]] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            try:
                mt = os.path.getmtime(p)
            except OSError:
                continue
            candidates.append((p, mt))
    if not candidates:
        return None
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0]

def _norm_ccy(x: Any) -> str:
    s = (x or "").strip().upper()
    return s if re.fullmatch(r"[A-Z]{3,4}", s) else ""

def _parse_rate(x: Any) -> Optional[float]:
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    sx = str(x).strip().replace(",", "")
    try:
        return float(sx)
    except ValueError:
        return None

def _to_mmddyyyy_from_iso(iso: str) -> str:
    # "YYYY-MM-DD" -> "MM/DD/YYYY"
    m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", iso)
    if not m:
        return ""
    yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
    return f"{mm}/{dd}/{yyyy}"

def _is_mmddyyyy(s: str) -> bool:
    return bool(re.fullmatch(r"\d{2}/\d{2}/\d{4}", s or ""))

def _key_ci(d: Dict[str, Any], *cands: str) -> Optional[str]:
    lm = {k.lower(): k for k in d.keys()}
    for c in cands:
        k = lm.get(c.lower())
        if k is not None:
            return k
    return None

# ---------- row normalization ----------
def normalize_row_from_csv(raw: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Map your CSV schema to:
    {
      "ExchangeRateType": "M",
      "FromCurrency": "...",
      "ToCurrency": "...",
      "ValidFrom": "MM/DD/YYYY",
      "Quotation": "Direct",
      "ExchangeRate": float
    }
    """
    # currencies: prefer FromCurrency/ToCurrency, else sourceCurrency/targetCurrency
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    # rate: exChangeRateValue (preferred), else ExchangeRate/Rate/Value
    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    # date: exChangeRateDate is already MM/DD/YYYY; else fallback to _service_date ISO
    k_date_csv = _key_ci(raw, "exChangeRateDate")
    valid_from = ""
    if k_date_csv:
        cand = (raw.get(k_date_csv) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = cand
    if not valid_from:
        k_iso = _key_ci(raw, "_service_date")  # YYYY-MM-DD
        if k_iso and raw.get(k_iso):
            valid_from = _to_mmddyyyy_from_iso(str(raw[k_iso]).strip())
    if not valid_from:
        return None

    return {
        "ExchangeRateType": "M",
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,  # MM/DD/YYYY
        "Quotation": "Direct",
        "ExchangeRate": rate,
    }

def normalize_row_from_json(raw: Dict[str, Any], fallback_iso_date: str = "") -> Optional[Dict[str, Any]]:
    # currencies
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    # rate
    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    # date: try exChangeRateDate (MM/DD/YYYY), ValidFrom/Date/RateDate/_service_date (may be ISO)
    k_mmdd = _key_ci(raw, "exChangeRateDate")
    k_other= _key_ci(raw, "ValidFrom", "Date", "RateDate", "_service_date")
    valid_from = ""
    if k_mmdd:
        cand = (raw.get(k_mmdd) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = cand
    if not valid_from and k_other and raw.get(k_other):
        iso_like = str(raw[k_other]).strip()
        if _is_mmddyyyy(iso_like):
            valid_from = iso_like
        else:
            valid_from = _to_mmddyyyy_from_iso(iso_like)
    if not valid_from and fallback_iso_date:
        valid_from = _to_mmddyyyy_from_iso(fallback_iso_date)
    if not valid_from:
        return None

    # defaults
    k_type = _key_ci(raw, "ExchangeRateType", "Type")
    k_quo  = _key_ci(raw, "Quotation", "QuoteType", "Side")
    etype = str(raw.get(k_type)).strip() if (k_type and raw.get(k_type)) else "M"
    quo   = str(raw.get(k_quo)).strip()  if (k_quo  and raw.get(k_quo))  else "Direct"

    return {
        "ExchangeRateType": etype,
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,  # MM/DD/YYYY
        "Quotation": quo,
        "ExchangeRate": rate,
    }

# ---------- loaders ----------
def load_from_filtered_csv(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    path = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")
    if not os.path.isfile(path):
        return []
    with open(path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        rows_raw = list(rdr)

    out: List[Dict[str, Any]] = []
    for r in rows_raw:
        nr = normalize_row_from_csv(r)
        if nr:
            out.append(nr)
        else:
            skip_stats["csv_skipped"] += 1
    return out

def load_from_filtered_jsons(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    base = os.path.join(run_dir, "exchange_rates_json_filtered")
    if not os.path.isdir(base):
        return []
    out: List[Dict[str, Any]] = []

    for name in os.listdir(base):
        if not name.lower().endswith(".json"):
            continue
        p = os.path.join(base, name)
        fallback_iso = ""
        m = FNAME_DATE_RE.match(name)
        if m:
            yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
            fallback_iso = f"{yyyy}-{mm}-{dd}"

        try:
            with open(p, "r", encoding="utf-8") as f:
                obj = json.load(f)
        except Exception:
            skip_stats["bad_json"] += 1
            continue

        if isinstance(obj, dict) and isinstance(obj.get("data"), list):
            iterable = obj["data"]
        elif isinstance(obj, list):
            iterable = obj
        elif isinstance(obj, dict):
            iterable = [obj]
        else:
            skip_stats["unknown_shape"] += 1
            continue

        for item in iterable:
            if isinstance(item, dict):
                nr = normalize_row_from_json(item, fallback_iso_date=fallback_iso)
                if nr:
                    out.append(nr)
                else:
                    skip_stats["json_row_skipped"] += 1
            else:
                skip_stats["non_dict_row"] += 1
    return out

def dedupe_sort(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # Deduplicate by (From, To, ValidFrom, Type, Quotation); keep last seen
    seen: Dict[Tuple[str, str, str, str, str], Dict[str, Any]] = {}
    for r in rows:
        key = (r["FromCurrency"], r["ToCurrency"], r["ValidFrom"], r["ExchangeRateType"], r["Quotation"])
        seen[key] = r
    uniq = list(seen.values())
    # Sort by date (MM/DD/YYYY) safely: convert to YYYYMMDD for sort
    def _key(r: Dict[str, Any]) -> Tuple[str, str, str]:
        m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", r["ValidFrom"])
        ymd = f"{m.group(3)}{m.group(1)}{m.group(2)}" if m else "00000000"
        return (ymd, r["FromCurrency"], r["ToCurrency"])
    uniq.sort(key=_key)
    return uniq

# ---------- main ----------
def build_payload() -> str:
    run_dir = pick_latest_run_dir(BASE_DIR)
    if not run_dir:
        raise SystemExit(f"No run directory under {BASE_DIR} matching YYYY-MM-DD__YYYY-MM-DD")

    print(f"[run-dir] {run_dir}")

    skip_stats = {
        "csv_skipped": 0,
        "bad_json": 0,
        "unknown_shape": 0,
        "json_row_skipped": 0,
        "non_dict_row": 0,
    }

    # Prefer filtered CSV with your exact headers; fallback to filtered JSONs
    rows = load_from_filtered_csv(run_dir, skip_stats)
    source = "CSV"
    if not rows:
        rows = load_from_filtered_jsons(run_dir, skip_stats)
        source = "JSON"

    rows = dedupe_sort(rows)
    print(f"[source] {source}, rows={len(rows)}; skips={skip_stats}")

    out_path = os.path.join(run_dir, "exchange_rates_payload.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(rows, f, ensure_ascii=False, indent=2)

    print(f"[written] {out_path}")
    return out_path

if __name__ == "__main__":
    build_payload()


[run-dir] WebService\2025-07-01__2025-07-01
[source] CSV, rows=591; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService\2025-07-01__2025-07-01\exchange_rates_payload.json
